In [2]:
from abc import abstractmethod
from typing import Optional
from typing import Dict
from typing import List
from typing import Union
import numpy as np
from tqdm import tqdm
from omegaconf import OmegaConf
from omegaconf import DictConfig
import hydra
from hydra.core.config_store import ConfigStore
import pytorch_lightning as pl

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from torch.utils.data import random_split
from torchvision import transforms
import wandb
from datetime import datetime
import sys
import os

In [3]:
sys.path.append("/root/pointnet.pytorch")

In [4]:
from utils.data_utils import dataset_split
from utils.config_utils import flatten_dict
from utils.config_utils import register_config
from utils.config_utils import configure_optimizers_from_cfg
from utils.config_utils import get_loggers
from utils.config_utils import get_callbacks
from utils.custom_math import softmax
from pointnet.dataset import ShapeNetDataset

In [5]:
class BaseLightningModule(pl.LightningModule):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.cfg = cfg
        # self.loss_function = nn.CrossEntropyLoss()

    @abstractmethod
    def forward(self, inputs, target):
        raise NotImplemented()

    def configure_optimizers(self):
        self._optimizer, self._schedulers = configure_optimizers_from_cfg(self.cfg, self)
        return self._optimizer, self._schedulers

    def _forward(self, points, target, mode: str):

        assert mode in ["train", "val", "test"]

        current_loss = 0.0
        current_corrects = 0

        # get predictions
        #outputs = self(images)
        target = target[:, 0]
        points = points.transpose(2, 1)

        pred, trans, trans_feat = model(points)

        # get loss(Loss 계산)
        loss = F.nll_loss(pred, target)
        pred_choice = pred.data.max(1)[1]
        corrects = torch.sum(pred_choice == target.data)
        acc = corrects / len(pred)

        return {
            f"{mode}_loss": loss,
            f"{mode}_acc": acc,
        }

    def training_step(self, batch, batch_idx):
        points, target = batch
        logs = self._forward(points, target, mode="train")
        self.log_dict(logs)
        logs["loss"] = logs["train_loss"]
        cul_lr = self._optimizer.param_groups[0]["lr"] if self._schedulers is None else self._schedulers[0].get_last_lr()[0]
        wandb.log({"learning_rate": cul_lr})

        return logs

    def validation_step(self, batch, batch_idx):
        points, target = batch
        logs = self._forward(points, target, mode="val")
        self.log_dict(logs)
        logs["loss"] = logs["val_loss"]
        return logs

    def test_step(self, batch, batch_idx):
        points, target = batch
        logs = self._forward(points, target, mode="test")
        self.log_dict(logs)
        logs["loss"] = logs["test_loss"]
        return logs

In [6]:
class PointNetCls(BaseLightningModule):
    def __init__(self, cfg: DictConfig):
        BaseLightningModule.__init__(self, cfg=cfg)
        
        self.feat = PointNetfeat(cfg)
        self.fc1 = nn.Linear(cfg.model.cls.layer1.in_feature, cfg.model.cls.layer1.out_feature)
        self.fc2 = nn.Linear(cfg.model.cls.layer2.in_feature, cfg.model.cls.layer2.out_feature)
        self.fc3 = nn.Linear(cfg.model.cls.layer3.in_feature, cfg.model.cls.layer3.out_feature)
        self.dropout = nn.Dropout(cfg.model.cls.dropout)
        self.bn1 = nn.BatchNorm1d(cfg.model.cls.layer1.out_feature)
        self.bn2 = nn.BatchNorm1d(cfg.model.cls.layer2.out_feature)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.dropout(self.fc2(x))))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1), trans, trans_feat

In [7]:
class PointNetDenseCls(BaseLightningModule):
    def __init__(self, cfg: DictConfig):
        BaseLightningModule.__init__(self, cfg=cfg)
        
        self.k = cfg.model.densecls.k
        self.feat = PointNetfeat(cfg)
        self.conv1 = torch.nn.Conv1d(
            cfg.model.densecls.layer1.conv1d_in_channel,
            cfg.model.densecls.layer1.conv1d_out_channel,
            cfg.model.densecls.layer1.conv1d_kernel_size
        )
        self.conv2 = torch.nn.Conv1d(
            cfg.model.densecls.layer2.conv1d_in_channel,
            cfg.model.densecls.layer2.conv1d_out_channel,
            cfg.model.densecls.layer2.conv1d_kernel_size
        )
        self.conv3 = torch.nn.Conv1d(
            cfg.model.densecls.layer3.conv1d_in_channel,
            cfg.model.densecls.layer3.conv1d_out_channel,
            cfg.model.densecls.layer3.conv1d_kernel_size
        )
        self.conv4 = torch.nn.Conv1d(
            cfg.model.densecls.layer4.conv1d_in_channel,
            self.k,
            cfg.model.densecls.layer3.conv1d_kernel_size
        )
        self.bn1 = nn.BatchNorm1d(cfg.model.densecls.layer1.conv1d_out_channel)
        self.bn2 = nn.BatchNorm1d(cfg.model.densecls.layer2.conv1d_out_channel)
        self.bn3 = nn.BatchNorm1d(cfg.model.densecls.layer3.conv1d_out_channel)

    def forward(self, x):
        batchsize = x.size()[0]
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans, trans_feat

In [8]:
class PointNetfeat(nn.Module):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.stn = STN3d(cfg)
        self.conv1 = torch.nn.Conv1d(
            cfg.model.feat.layer1.conv1d_in_channel,
            cfg.model.feat.layer1.conv1d_out_channel,
            cfg.model.feat.layer1.conv1d_kernel_size
        )
        self.conv2 = torch.nn.Conv1d(
            cfg.model.feat.layer2.conv1d_in_channel,
            cfg.model.feat.layer2.conv1d_out_channel,
            cfg.model.feat.layer2.conv1d_kernel_size
        )
        self.conv3 = torch.nn.Conv1d(
            cfg.model.feat.layer3.conv1d_in_channel,
            cfg.model.feat.layer3.conv1d_out_channel,
            cfg.model.feat.layer3.conv1d_kernel_size
        )
        self.bn1 = nn.BatchNorm1d(cfg.model.feat.layer1.conv1d_out_channel)
        self.bn2 = nn.BatchNorm1d(cfg.model.feat.layer2.conv1d_out_channel)
        self.bn3 = nn.BatchNorm1d(cfg.model.feat.layer3.conv1d_out_channel)
        self.global_feat = cfg.model.feat.global_feat
        self.feature_transform = cfg.model.feat.feature_transform
        if self.feature_transform:
            self.fstn = STNkd(cfg)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))

        x = torch.max(x, 2, keepdim=True)[0]

        x = x.view(-1, 1024)

        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat

In [9]:
class STN3d(nn.Module):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.conv1 = torch.nn.Conv1d(
            cfg.model.stn.layer1.conv1d_in_channel,
            cfg.model.stn.layer1.conv1d_out_channel,
            cfg.model.stn.layer1.conv1d_kernel_size
        )

        self.conv2 = torch.nn.Conv1d(
            cfg.model.stn.layer2.conv1d_in_channel,
            cfg.model.stn.layer2.conv1d_out_channel,
            cfg.model.stn.layer2.conv1d_kernel_size
        )
    
        self.conv3 = torch.nn.Conv1d(
            cfg.model.stn.layer3.conv1d_in_channel,
            cfg.model.stn.layer3.conv1d_out_channel,
            cfg.model.stn.layer3.conv1d_kernel_size
        )
    
        self.fc1 = nn.Linear(cfg.model.stn.layer4.fc1_in_features, cfg.model.stn.layer4.fc1_out_features)
        self.fc2 = nn.Linear(cfg.model.stn.layer5.fc2_in_features, cfg.model.stn.layer5.fc2_out_features)
        self.fc3 = nn.Linear(cfg.model.stn.layer6.fc3_in_features, cfg.model.stn.layer6.fc3_out_features)
    
        self.bn1 = nn.BatchNorm1d(cfg.model.stn.layer1.conv1d_out_channel)
        self.bn2 = nn.BatchNorm1d(cfg.model.stn.layer2.conv1d_out_channel)
        self.bn3 = nn.BatchNorm1d(cfg.model.stn.layer3.conv1d_out_channel)
        self.bn4 = nn.BatchNorm1d(cfg.model.stn.layer4.fc1_out_features)
        self.bn5 = nn.BatchNorm1d(cfg.model.stn.layer5.fc2_out_features)
        
        relu = nn.ReLU()


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,.0,0,1]).astype(np.float32)))view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x

In [10]:
class STNkd(nn.Module):
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.k = cfg.model.stn.k
        
        self.conv1 = torch.nn.Conv1d(
            self.k,
            cfg.model.stn.layer1.conv1d_out_channel,
            cfg.model.stn.layer1.conv1d_kernel_size
        )

        self.conv2 = torch.nn.Conv1d(
            cfg.model.stn.layer2.conv1d_in_channel,
            cfg.model.stn.layer2.conv1d_out_channel,
            cfg.model.stn.layer2.conv1d_kernel_size
        )
    
        self.conv3 = torch.nn.Conv1d(
            cfg.model.stn.layer3.conv1d_in_channel,
            cfg.model.stn.layer3.conv1d_out_channel,
            cfg.model.stn.layer3.conv1d_kernel_size
        )
    
        self.fc1 = nn.Linear(cfg.model.stn.layer4.fc1_in_features, cfg.model.stn.layer4.fc1_out_features)
        self.fc2 = nn.Linear(cfg.model.stn.layer5.fc2_in_features, cfg.model.stn.layer5.fc2_out_features)
        self.fc3 = nn.Linear(cfg.model.stn.layer6.fc3_in_features, self.k*self.k)
    
        self.bn1 = nn.BatchNorm1d(cfg.model.stn.layer1.conv1d_out_channel)
        self.bn2 = nn.BatchNorm1d(cfg.model.stn.layer2.conv1d_out_channel)
        self.bn3 = nn.BatchNorm1d(cfg.model.stn.layer3.conv1d_out_channel)
        self.bn4 = nn.BatchNorm1d(cfg.model.stn.layer4.fc1_out_features)
        self.bn5 = nn.BatchNorm1d(cfg.model.stn.layer5.fc2_out_features)
        
        self.relu = nn.ReLU()


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x


In [11]:
# model configs
model_pointnet_cfg = {
    "name": "PointNet",
    "cls": {
        "layer1":{
            "in_feature": 1024,
            "out_feature": 512,
        },
        "layer2":{
            "in_feature": 512,
            "out_feature": 256,
        },
        "layer3":{
            "in_feature": 256,
            "out_feature": 16, 
        },
        "dropout" : 0.3,

    },
    "densecls": {
        "k" : 16,  ########
        "layer1": {
            "conv1d_in_channel": 1088,
            "conv1d_out_channel": 512,
            "conv1d_kernel_size": 1,
        },
        "layer2": {
            "conv1d_in_channel": 512,
            "conv1d_out_channel": 256,
            "conv1d_kernel_size": 1,
        },
        "layer3": {
            "conv1d_in_channel": 256,
            "conv1d_out_channel": 128,
            "conv1d_kernel_size": 1,
        },
        "layer4":{
            "conv1d_in_channel": 128,
            "conv1d_kernel_size": 1,
        },
    },
    "feat":{
        "layer1": {
            "conv1d_in_channel": 3,
            "conv1d_out_channel": 64,
            "conv1d_kernel_size": 1,
        },
         "layer2": {
            "conv1d_in_channel": 64,
            "conv1d_out_channel": 128,
            "conv1d_kernel_size": 1,
        },
         "layer3": {
            "conv1d_in_channel": 128,
            "conv1d_out_channel": 1024,
            "conv1d_kernel_size": 1,
        },
        "global_feat": True,
        "feature_transform": True,
    },
    "stn": {
        "k": 64, 
        "layer1": {
          "conv1d_in_channel": 3,
          "conv1d_out_channel": 64,
          "conv1d_kernel_size": 1,
        },
        "layer2": {
          "conv1d_in_channel": 64,
          "conv1d_out_channel": 128,
          "conv1d_kernel_size": 1,
        },
        "layer3": {
          "conv1d_in_channel": 128,
          "conv1d_out_channel": 1024,
          "conv1d_kernel_size": 1,
        },
        "layer4": {
          "fc1_in_features": 1024,
          "fc1_out_features": 512,
        },
        "layer5": {
          "fc2_in_features": 512,
          "fc2_out_features": 256,
        },
        "layer6": {
          "fc3_in_features": 256,
          "fc3_out_features": 9,
        }  
    }
}

        
# optimizer configs
opt_cfg = {
    "optimizers": [
        {
            "name": "RAdam",
            "kwargs": {
                "lr": 1e-3,
            }
        }
    ],
    "lr_schedulers": [
        {
            "name": "LinearWarmupLR",
            "kwargs": {
                "warm_end_steps": 1000
            }
        },
    ]
}

data_shapenet_cfg = {
    "name" : "shapenet",
    "data_root" : os.path.join(os.getcwd(), "data"),
    "num_points" : 2500
}

_merged_cfg_presets = {
    "PointNet": {
        "opt": opt_cfg,
        "data": data_shapenet_cfg,
        "model": model_pointnet_cfg,
    },
}

# clear config instance first
hydra.core.global_hydra.GlobalHydra.instance().clear()

# register preset configs
register_config(_merged_cfg_presets)

# initialize & mae config
## select mode here ##
# .................. #
hydra.initialize(config_path=None)
cfg = hydra.compose("PointNet")

# override some cfg
run_name = f"{datetime.now().isoformat(timespec='seconds')}-{cfg.model.name}-{cfg.data.name}"

# Define other train configs & log_configs
# Merge configs into one & register it to Hydra.

project_root_dir = os.path.join(
    os.getcwd(), "runs", "pointnet-runs"
)
save_dir = os.path.join(project_root_dir, run_name)
run_root_dir = os.path.join(project_root_dir, run_name)

train_cfg = {
    "train_batch_size": 32,
    "val_batch_size": 16,
    "test_batch_size": 32,
    "train_val_split": [0.9, 0.1],
    "run_root_dir": run_root_dir,
    "s": {
        "accelerator": "auto",
        "max_epochs": 50,
        "val_check_interval": 1.0,
        "log_every_n_steps": 100,
    }
}

# logger config
log_cfg = {
    "loggers": {
        "WandbLogger": {
            "project": "pointNet",
            "name": run_name,
            "tags": ["fastcampus_de_en_translate_tutorials"],
            "save_dir": run_root_dir,
        },
    },
    "callbacks": {
        "ModelCheckpoint": {
            "save_top_k": 3,
            "monitor": "val_loss",
            "mode": "min",
            "verbose": True,
            "dirpath": os.path.join(run_root_dir, "weights"),
            "filename": "{epoch}-{val_loss:.3f}",
        },
        "EarlyStopping": {
            "monitor": "val_loss",
            "mode": "min",
            "patience": 3,
            "verbose": True,
        }
    }
}

# unlock config & set train_cfg & log_cfg
OmegaConf.set_struct(cfg, False)
cfg.train = train_cfg
cfg.log = log_cfg

# lock config
OmegaConf.set_struct(cfg, True)
print(OmegaConf.to_yaml(cfg))

opt:
  optimizers:
  - name: RAdam
    kwargs:
      lr: 0.001
  lr_schedulers:
  - name: LinearWarmupLR
    kwargs:
      warm_end_steps: 1000
data:
  name: shapenet
  data_root: /root/pointnet.pytorch/data
  num_points: 2500
model:
  name: PointNet
  cls:
    layer1:
      in_feature: 1024
      out_feature: 512
    layer2:
      in_feature: 512
      out_feature: 256
    layer3:
      in_feature: 256
      out_feature: 16
    dropout: 0.3
  densecls:
    k: 16
    layer1:
      conv1d_in_channel: 1088
      conv1d_out_channel: 512
      conv1d_kernel_size: 1
    layer2:
      conv1d_in_channel: 512
      conv1d_out_channel: 256
      conv1d_kernel_size: 1
    layer3:
      conv1d_in_channel: 256
      conv1d_out_channel: 128
      conv1d_kernel_size: 1
    layer4:
      conv1d_in_channel: 128
      conv1d_kernel_size: 1
  feat:
    layer1:
      conv1d_in_channel: 3
      conv1d_out_channel: 64
      conv1d_kernel_size: 1
    layer2:
      conv1d_in_channel: 64
      conv1d_out_chan

/tmp/ipykernel_87119/211539327.py:137: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_path=None)


In [12]:
data_root = cfg.data.data_root

dataset = ShapeNetDataset(
    root=data_root, 
    classification=True, 
    npoints=cfg.data.num_points
)

test_dataset = ShapeNetDataset(
    root=data_root,
    classification=True,
    split='test',
    npoints=cfg.data.num_points,
    data_augmentation=False
)

num_classes = len(dataset.classes)
print(num_classes)

dataset = dataset_split(dataset, split=cfg.train.train_val_split)
train_dataset = dataset["train"]
val_dataset = dataset["val"]

train_batch_size = cfg.train.train_batch_size
val_batch_size = cfg.train.val_batch_size
test_batch_size = cfg.train.test_batch_size

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=0
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_batch_size, shuffle=True, num_workers=0
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=0
)


{'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5, 'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10, 'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15}
{'Airplane': 4, 'Bag': 2, 'Cap': 2, 'Car': 4, 'Chair': 4, 'Earphone': 3, 'Guitar': 3, 'Knife': 2, 'Lamp': 4, 'Laptop': 2, 'Motorbike': 6, 'Mug': 2, 'Pistol': 3, 'Rocket': 3, 'Skateboard': 3, 'Table': 3} 4
{'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5, 'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10, 'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15}
{'Airplane': 4, 'Bag': 2, 'Cap': 2, 'Car': 4, 'Chair': 4, 'Earphone': 3, 'Guitar': 3, 'Knife': 2, 'Lamp': 4, 'Laptop': 2, 'Motorbike': 6, 'Mug': 2, 'Pistol': 3, 'Rocket': 3, 'Skateboard': 3, 'Table': 3} 4
16


In [13]:
# model define

def get_pl_model(cfg: DictConfig, checkpoint_path: Optional[str] = None):

    if cfg.model.name == "PointNet":
        model = PointNetCls(cfg)
    else:
        raise NotImplementedError()

    if checkpoint_path is not None:
        model = PLMLP.load_from_checkpoint(checkpoint_path)

    return model

model = get_pl_model(cfg)
print(model)

PointNetCls(
  (feat): PointNetfeat(
    (stn): STN3d(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128,

In [14]:
logger = get_loggers(cfg)
callbacks = get_callbacks(cfg)

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir=cfg.train.run_root_dir,
    num_sanity_val_steps=2,
    **cfg.train.trainer_kwargs
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [15]:
trainer.fit(model, train_dataloader, val_dataloader)

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params | Mode 
-------------------------------------------------
0 | feat    | PointNetfeat | 2.8 M  | train
1 | fc1     | Linear       | 524 K  | train
2 | fc2     | Linear       | 131 K  | train
3 | fc3     | Linear       | 4.1 K  | train
4 | dropout | Dropout      | 0      | train
5 | bn1     | BatchNorm1d  | 1.0 K  | train
6 | bn2     | BatchNorm1d  | 512    | train
7 | relu    | ReLU         | 0      | train
-------------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
13.861    Total estimated model params size (MB)
39        Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/opt/conda/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


...

Epoch 26: 100%|██████████| 342/342 [00:28<00:00, 12.04it/s, v_num=f4yb]

Epoch 26, global step 9234: 'val_loss' reached 0.11057 (best 0.07191), saving model to '/root/pointnet.pytorch/runs/pointnet-runs/2025-01-13T16:39:57-PointNet-shapenet/weights/epoch=26-val_loss=0.111.ckpt' as top 3


Epoch 27: 100%|██████████| 342/342 [00:28<00:00, 12.05it/s, v_num=f4yb]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.072. Signaling Trainer to stop.
Epoch 27, global step 9576: 'val_loss' reached 0.08747 (best 0.07191), saving model to '/root/pointnet.pytorch/runs/pointnet-runs/2025-01-13T16:39:57-PointNet-shapenet/weights/epoch=27-val_loss=0.087.ckpt' as top 3


Epoch 27: 100%|██████████| 342/342 [00:28<00:00, 12.01it/s, v_num=f4yb]


In [16]:
# model configs
model_pointnet_cfg = {
    "name": "PointNet",
    "cls": {
        "layer1":{
            "in_feature": 1024,
            "out_feature": 512,
        },
        "layer2":{
            "in_feature": 512,
            "out_feature": 256,
        },
        "layer3":{
            "in_feature": 256,
            "out_feature": 16, 
        },
        "dropout" : 0.3,

    },
    "densecls": {
        "k" : 16,  ########
        "layer1": {
            "conv1d_in_channel": 1088,
            "conv1d_out_channel": 512,
            "conv1d_kernel_size": 1,
        },
        "layer2": {
            "conv1d_in_channel": 512,
            "conv1d_out_channel": 256,
            "conv1d_kernel_size": 1,
        },
        "layer3": {
            "conv1d_in_channel": 256,
            "conv1d_out_channel": 128,
            "conv1d_kernel_size": 1,
        },
        "layer4":{
            "conv1d_in_channel": 128,
            "conv1d_kernel_size": 1,
        },
    },
    "feat":{
        "layer1": {
            "conv1d_in_channel": 3,
            "conv1d_out_channel": 64,
            "conv1d_kernel_size": 1,
        },
         "layer2": {
            "conv1d_in_channel": 64,
            "conv1d_out_channel": 128,
            "conv1d_kernel_size": 1,
        },
         "layer3": {
            "conv1d_in_channel": 128,
            "conv1d_out_channel": 1024,
            "conv1d_kernel_size": 1,
        },
        "global_feat": True,
        "feature_transform": True,
    },
    "stn": {
        "k": 64, 
        "layer1": {
          "conv1d_in_channel": 3,
          "conv1d_out_channel": 64,
          "conv1d_kernel_size": 1,
        },
        "layer2": {
          "conv1d_in_channel": 64,
          "conv1d_out_channel": 128,
          "conv1d_kernel_size": 1,
        },
        "layer3": {
          "conv1d_in_channel": 128,
          "conv1d_out_channel": 1024,
          "conv1d_kernel_size": 1,
        },
        "layer4": {
          "fc1_in_features": 1024,
          "fc1_out_features": 512,
        },
        "layer5": {
          "fc2_in_features": 512,
          "fc2_out_features": 256,
        },
        "layer6": {
          "fc3_in_features": 256,
          "fc3_out_features": 9,
        }  
    }
}

        
# optimizer configs
opt_cfg = {
    "optimizers": [
        {
            "name": "RAdam",
            "kwargs": {
                "lr": 1e-3,
            }
        }
    ],
    "lr_schedulers": [
        {
            "name": "LinearWarmupLR",
            "kwargs": {
                "warm_end_steps": 1000
            }
        },
    ]
}

data_shapenet_cfg = {
    "name" : "shapenet",
    "data_root" : os.path.join(os.getcwd(), "data"),
    "num_points" : 2500
}

_merged_cfg_presets = {
    "PointNet": {
        "opt": opt_cfg,
        "data": data_shapenet_cfg,
        "model": model_pointnet_cfg,
    },
}


In [18]:
import yaml
import os

def save_yaml(cfg, filename):
    with open(filename, 'w') as file:
        yaml.dump(cfg, file, default_flow_style=False, sort_keys=False, allow_unicode=True)

# 저장할 파일 경로
output_dir = "./config_yaml"
os.makedirs(output_dir, exist_ok=True)

# 각각 YAML로 저장
save_yaml(model_pointnet_cfg, os.path.join(output_dir, "model_pointnet.yaml"))
save_yaml(opt_cfg, os.path.join(output_dir, "optimizer.yaml"))
save_yaml(data_shapenet_cfg, os.path.join(output_dir, "data_shapenet.yaml"))
save_yaml(_merged_cfg_presets, os.path.join(output_dir, "merged_cfg_presets.yaml"))

In [ ]:
train_cfg = {
    "train_batch_size": 32,
    "val_batch_size": 16,
    "test_batch_size": 32,
    "train_val_split": [0.9, 0.1],
    "run_root_dir": run_root_dir,
    "trainer_kwargs": {
        "accelerator": "auto",
        "max_epochs": 50,
        "val_check_interval": 1.0,
        "log_every_n_steps": 100,
    }
}

# logger config
log_cfg = {
    "loggers": {
        "WandbLogger": {
            "project": "pointNet",
            "name": run_name,
            "tags": ["fastcampus_de_en_translate_tutorials"],
            "save_dir": run_root_dir,
        },
    },
    "callbacks": {
        "ModelCheckpoint": {
            "save_top_k": 3,
            "monitor": "val_loss",
            "mode": "min",
            "verbose": True,
            "dirpath": os.path.join(run_root_dir, "weights"),
            "filename": "{epoch}-{val_loss:.3f}",
        },
        "EarlyStopping": {
            "monitor": "val_loss",
            "mode": "min",
            "patience": 3,
            "verbose": True,
        }
    }
}


In [19]:
import yaml
import os

def save_yaml(cfg, filename):
    with open(filename, 'w') as file:
        yaml.dump(cfg, file, default_flow_style=False, sort_keys=False, allow_unicode=True)

# 저장할 파일 경로
output_dir = "./config_yaml"
os.makedirs(output_dir, exist_ok=True)

# 각각 YAML로 저장
save_yaml(train_cfg, os.path.join(output_dir, "train.yaml"))
save_yaml(log_cfg, os.path.join(output_dir, "log.yaml"))